In [1]:
import os
import yaml

import polars as pl
import markdown

from io import StringIO

from bs4 import BeautifulSoup

In [2]:
paths = []
for root, _, files in os.walk("poki/plaintext"):
    for file in files:
        paths.append(os.path.join(root, file))

len(paths)

1502

In [3]:
def markdown_to_text(md: str) -> str:
    html = markdown.markdown(md)
    soup = BeautifulSoup(html, "html.parser")
    text = soup.get_text().replace("\\\n", "\n")
    return text

In [4]:
def load_md(path):
    with open(path) as f:
        content = f.read()

    meta = content.split("---\n")[1]
    meta = yaml.safe_load(StringIO(meta))

    text = content.split("---\n")[2]
    text = markdown_to_text(text)
    return meta | {"text": text}

In [5]:
all_keys = set()
data = []

for path in paths:
    post = load_md(path)
    for key in post.keys():
        all_keys.add(key)

    data.append(post)

for row in data:
    for key in all_keys:
        if key not in row:
            row[key] = None

    if not isinstance(row["date"], str):
        row["date"] = str(row["date"])

In [6]:
df = pl.DataFrame(
    data,
    infer_schema_length=len(data)
)
df

title,description,authors,proofreaders,date,date-precision,tags,original,license,sources,archives,preprocessing,accessibility-notes,notes,text
str,str,list[str],list[str],str,str,list[str],struct[2],str,list[str],list[str],str,str,str,str
"""lipu Tili: lipu nanpa wan""",null,"[""soweli Peka""]",null,"""2013-04-18""","""day""",null,null,null,"[""https://tokisoweli.blogspot.com/2013/04/pu-tili-pi-pu-lili-wan.html""]",null,null,null,null,"""lon li ike tawa jan mute. kulu…"
"""lipu Tili: lipu nanpa tu""",null,"[""soweli Peka""]",null,"""2013-04-21""","""day""",null,null,null,"[""https://tokisoweli.blogspot.com/2013/04/pu-tili-pu-lili-tu.html""]",null,null,null,null,"""suno li kama mute li tawa mute…"
"""lipu Tili: lipu nanpa tu wan""",null,"[""soweli Peka""]",null,"""2013-04-22""","""day""",null,null,null,"[""https://tokisoweli.blogspot.com/2013/04/pu-lili-tu-wan.html""]",null,null,null,null,"""jan Tili li jo e sona utala ta…"
"""kalama musi pi jan sewi meli -…","""kalama musi Pekan li pona mute…","[""jan Minasa""]",null,"""2013-10-21""","""day""","[""music""]",null,null,"[""https://www.youtube.com/watch?v=IW1GkHioQ6Y""]",null,null,null,null,"""mama pimeja, mama suno ma pi l…"
"""telo nasa""",null,"[""tobiah""]",null,"""2013-10-10""","""day""","[""original"", ""lyrics"", … ""synth""]",null,"""CC BY-SA 3.0""","[""https://tobiah.bandcamp.com/track/telo-nasa""]",null,null,null,null,"""telo nasa li pona. telo nasa…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""kili lili""",null,"[""jan Pije""]",null,"""2002-10-10""","""day""","[""original"", ""poetry""]",null,"""CC0 1.0""","[""http://tokipona.net/tp/janpije/kililili.php"", ""http://forums.tokipona.org/viewtopic.php?t=71#p177""]","[""https://web.archive.org/web/20151001204355/http://tokipona.net/tp/janpije/kililili.php"", ""https://wikisource.org/wiki/Kili_lili""]",null,null,null,"""mi jo e kili. ona li pona li…"
"""wan taso""","""Even angst-ridden, mediocre te…","[""jan Sonja""]",null,"""2003-04-23""","""day""","[""original"", ""poetry"", ""angst""]",null,"""All Rights Reserved""","[""http://www.tokipona.org/lit-angst.php""]","[""https://archive.is/b9Sjt"", ""http://web.archive.bibalex.org/web/20030423115357/http://www.tokipona.org/lit-angst.php"", ""https://archive.is/o/b9Sjt/web.archive.bibalex.org/web/20030423115357/http://www.tokipona.org/kalama/wantaso.mp3""]",null,null,null,"""wan taso ijo li moku e mi. m…"
"""pilin ike""","""Even angst-ridden, mediocre te…","[""jan Sonja""]",null,"""2003-04-23""","""day""","[""original"", ""poetry"", ""angst""]",null,"""All Rights Reserved""","[""http://www.tokipona.org/lit-angst.php""]","[""https://archive.is/b9Sjt"", ""http://web.archive.bibalex.org/web/20030423115357/http://www.tokipona.org/lit-angst.php"", ""https://archive.is/o/b9Sjt/web.archive.bibalex.org/web/20030423115357/http://www.tokipona.org/kalama/pilinike.mp3""]",null,null,null,"""pilin ike mi lon pimeja. was…"


In [7]:
df.filter(df["original"].is_not_null())

title,description,authors,proofreaders,date,date-precision,tags,original,license,sources,archives,preprocessing,accessibility-notes,notes,text
str,str,list[str],list[str],str,str,list[str],struct[2],str,list[str],list[str],str,str,str,str
"""pakala - kalama musi 'When I'm…","""kalama musi 'When I'm Gone' li…","[""jan Minasa""]",null,"""2013-10-27""","""day""","[""music""]","{""Cups"",[""Anna Kendrick""]}",null,"[""https://www.youtube.com/watch?v=Lm87rp2Zq_s""]",null,null,null,null,"""mi pilin e ni: kalama musi 'te…"
"""jan lawa lili""","""A translation of Antoine de Sa…","[""Micheal F.""]",null,"""2013-01-20""","""day""","[""translation"", ""story"", ""prose""]","{""The Little Prince"",[""Antoine de Saint-Exupéry""]}","""CC BY-NC 4.0""","[""http://failbluedot.com/toki_pona/jan_lawa_lili/chap01""]","[""https://web.archive.org/web/20151009153200/http://failbluedot.com/toki_pona/jan_lawa_lili/chap01""]",null,null,"""The older translation""","""jan lawa lili 01 tenpo pi lili…"
"""wile""","""Performed for my NCEA Level 1 …","[""jan Minasa""]",null,"""2014-10-23""","""day""","[""music""]","{""Życzenie"",[""Stefan Witwicki""]}",null,"[""https://www.youtube.com/watch?v=TRTKqhyvxNw""]",null,null,null,null,"""mi suno jelo pi lon sewi laso …"
"""PoPiPo""","""if the quality of this cover s…","[""flower radio""]",null,"""2022-03-06""","""day""","[""music""]","{""PoPiPo"",[""Lamaze-P""]}",null,"[""https://www.youtube.com/watch?v=lHTNpca_mxM""]",null,null,null,null,"""li moku sina pilin pona telo l…"
"""o pona e ijo pakala""","""len pi jan Sijan li kama pakal…","[""jan Kita""]",null,"""2022-05-14""","""day""","[""StoryWeaver level 4""]","{""A Stitch in Time"",[""Himadri Das"", ""Veena Prasad""]}","""CC-BY-4.0""","[""https://storyweaver.org.in/en/stories/442898-o-pona-e-ijo-pakala""]",null,"""removed manual line breaks""",null,null,""" *mu pakala* ""ike a"" · jan Sij…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Toki Sewi Telo Suli (Prayer to…","""My first music video for my so…","[""Marine Guitars""]",null,"""2016-10-23""","""day""","[""music""]","{null,null}",null,"[""https://www.youtube.com/watch?v=61O1u0Igbcw""]",null,null,null,null,"""Suli telo li moku Suli telo li…"
"""ilo toki""",null,"[""jan Tepan""]",null,"""2016-09-25""","""day""","[""translation"", ""poetry""]","{""Телефон"",[""Korney Chukovsky""]}",null,"[""https://github.com/stefichjo/toki-pona/blob/master/musi/ilo-toki.md""]","[""https://web.archive.org/web/20191109180014/https://github.com/stefichjo/toki-pona/blob/master/musi/ilo-toki.md""]",null,null,null,"""ilo toki ilo toki mi li kama …"
"""ma tomo Pape""",null,"[""jan Pije""]",null,"""2005-07-15""","""day""",null,"{""Tower of Babel"",null}","""CC BY-SA 3.0""","[""https://en.wikipedia.org/w/index.php?title=Toki_Pona&oldid=18194572"", ""https://olukin.blogspot.com/2011/04/ma-tomo-pape.html""]",null,null,null,null,"""jan ali li kepeken e toki sama…"


In [8]:
df.write_parquet("poki.pq")